# Ejemplo de R-Arcgis-Bridge con Datos Raster

Ejemplo del uso de r-arcgis-bridge con datos raster. Se realiza la exploración de un fichero raster y una clasificación no supervisada con K-means. 

![Use Kernel R](https://www.r-project.org/Rlogo.png) Use Kernel R

In [ ]:
library(arcgisbinding)
library(raster)
library(cluster)
library(leaflet)
library(leaflet.esri)
library(ggplot2)
library(dplyr)
library(sp) 
arc.check_product()


In [ ]:
working_directory <- "C:\\esri\\r-arcgis-tools\\datasets"
setwd(working_directory)
path_raster_entrada <- paste(working_directory,"\\cartagena.gdb\\cartagena",sep="")
path_raster_img <- "c:\\esri\\env\\rastersalida.img"

In [ ]:
#Leer raster de entrada
obj_data <- arc.open(path_raster_entrada)
#Convertirlo en un objeto r-arcgis
arc_raster <- arc.raster(obj_data)
#Convertirlo en un objeto raster
mi_raster <- as.raster(arc_raster)

In [ ]:
mi_raster
dim(mi_raster)
ras <- stack(mi_raster)
ras

# Exploracion de Datos

In [ ]:
banda_2 <- ras@layers[2][[1]]
banda_2


In [ ]:
banda5 <- subset(ras,5)
banda5

In [ ]:
hist(getValues(banda5))


## Exploracion de datos con ggplot2

In [ ]:
## Filtrado de valores digitales para no ver los ceros
vals_hist <- data.frame(x=getValues(banda_2))
vals_hist <- filter(vals_hist,x>0)

## Dibujando
figura1 <- ggplot(vals_hist,aes(x=x)) + geom_histogram(fill="darkblue")
figura1

# Combinaciones de bandas

[Referencia de combinaciones de bandas para landsat 8](https://www.esri.com/arcgis-blog/products/product/imagery/band-combinations-for-landsat-8/)

## Color Natural 


In [ ]:
plotRGB(mi_raster,r=4,g=3,b=2)

In [ ]:
plotRGB(mi_raster,r=4,g=3,b=2, stretch="hist")

# Falso color (urbano)	7 6 4

In [ ]:
plotRGB(mi_raster,r=7,g=6,b=4, stretch="hist")

## Color infrarrojo (vegetación)	5 4 3

In [ ]:
plotRGB(mi_raster,r=5,g=4,b=3, stretch="hist")

### Agricultura

In [ ]:
plotRGB(mi_raster,r=6,g=5,b=2, stretch="hist")

### Penetracion Atmosférica

In [ ]:
plotRGB(mi_raster,r=7,g=6,b=5, stretch="hist")

In [ ]:
#Análisis de vegetación	6 5 4
plotRGB(mi_raster,r=6,g=5,b=4, stretch="hist")

## Calculo del Indice de vegetación NVDI
[Véase](https://pro.arcgis.com/es/pro-app/help/data/imagery/ndvi-function.htm) 


$NVDI = \frac{BNIR - BR}{BNIR + BR} $


In [ ]:
banda5 <- subset(ras,5)
banda4 <- subset(ras,4)
nvdi = (banda5-banda4)/(banda4+banda5)

graficaNVDI <- leaflet() %>% 
            addEsriBasemapLayer(esriBasemapLayers$Imagery) %>%
            addRasterImage(nvdi, colors = "Spectral", opacity=0.8)



In [ ]:
graficaNVDI

### Uso del Algoritmo de clasificación K-means

In [ ]:
#kmean calssification
?kmeans


v <- getValues(mi_raster)
i <- which(!is.na(v))
v <- na.omit(v)

E <- kmeans(v,5,iter.max = 100, nstart = 10)
kmeans_raster <- raster(mi_raster)
kmeans_raster[i] <- E$cluster
kmeans_raster

In [ ]:
pal <- colorNumeric(c("green", "red"), 1:5)
ImgClusters <- leaflet() %>% 
            addProviderTiles(providers$Esri) %>% 
            addRasterImage(kmeans_raster, colors = pal, opacity="0.5")


In [ ]:
ImgClusters

### Guardando archivo

Este paso escribe el resultado como fichero .img

In [ ]:
writeRaster(kmeans_raster, path_raster_img, format="HFA")


Este paso guarda el raster en una database ESRI

In [ ]:
path_raster_kmeans <- paste(working_directory,"\\cartagena.gdb\\cartagena_kmeans",sep="")
arc.write(path_raster_kmeans, kmeans_raster)

Algo Mas

In [ ]:
cp <- kmeans_raster
raw <- (kmeans_raster@data@values == 1) 
vects <- as.numeric(raw)
cp@data@values<-vects
pal <- colorNumeric(c("green", "white"), 1:2)
ImgMonte <- leaflet() %>% 
            addProviderTiles(providers$Esri) %>% 
            addRasterImage(cp ,  colors = pal, opacity=0.5)

In [ ]:
ImgMonte

Calculando area aproximada de ¿Mangles??

In [ ]:
vects[is.na(vects)] <- 0
ncells<- sum(vects)
area_monte <- (ncells*30*30)/1000000
area_monte